In [ ]:
#RAL

import openai
import pandas as pd
import numpy as np
import json
import time
import csv
import itertools
import os 
import logging

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

#####################################
## GLOBAL CONSTANTS ##
with open('../../creds/openai_creds.json', 'r') as f:
    data = json.load(f)
RANDOM_SEED = 416
API_KEY = data['api_key']
example_df = pd.read_csv("../../data/gt_main2.csv")
AUT_ITEMS = ["brick"]
PROMPTS = {
    "zero_shot": "What are creative uses for [OBJECT_NAME]? The goal is to come up with a creative idea, which is an idea that strikes people as clever, unusual, interesting, uncommon, humorous, innovative, or different. List [N] creative uses for [OBJECT_NAME]",

    "implicit": "What are creative uses for [OBJECT_NAME]? Here are example creative uses: [EXAMPLES] Based on the examples, list [N] creative uses for [OBJECT_NAME] that sounds like the examples.",

    "explicit": "What are creative uses for [OBJECT_NAME]? Here are example creative uses: [EXAMPLES] Carefully study the examples and their style, then list [N] creative uses for [OBJECT_NAME] that resemble the given examples. Match the style, length, and complexity of the creative ideas in the examples",
}
#####################################


## LOGGER ##
#####################################
if '__file__' in globals():
    log_file = os.path.splitext(os.path.basename(__file__))[0] + '.log'
else:
    log_file = 'logfile.log'
logging.basicConfig(filename=log_file, level=logging.INFO, filemode='w', format='%(asctime)s %(message)s')
#####################################


#####################################
def handle_prompt(prompt_base, object_name, examples, n_examples, temperature, frequency_penalty, presence_penalty):
    prompt = make_prompt(prompt_base, object_name, examples, n_examples)
    response = generate_responses_with_rate_limit_handling(prompt, temperature, frequency_penalty, presence_penalty)
    print(response)
    return response

def make_prompt(prompt_base, object_name, examples, n_examples):
    prompt = prompt_base.replace("[OBJECT_NAME]", object_name)
    prompt = prompt.replace("[N]", str(n_examples))
    examples = " ".join(['\n- ' + item for item in examples]) + "\n"
    prompt = prompt.replace("[EXAMPLES]", examples)
    return prompt

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def generate_responses(prompt, temperature, frequency_penalty, presence_penalty):
    openai.api_key = API_KEY
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=2000,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        temperature=temperature,
    )
    message = response["choices"][0]["text"].strip()
    return message


def get_examples(df, prompt, n_examples, seed=416):
    return df[df['prompt'] == prompt].sample(n_examples, random_state=seed)['response'].tolist()

def generate_responses_with_rate_limit_handling(prompt, temperature, frequency_penalty, presence_penalty):
    while True:
        try:
            return generate_responses(prompt, temperature, frequency_penalty, presence_penalty)
        except OpenAIAPIRateLimitError as e:
            print(f"Rate limit exceeded. Retrying in {e.seconds_to_wait} seconds...")
            time.sleep(e.seconds_to_wait)

def split_ideas(x):
    x = x.split("\n")
    x = [(l.replace("- ", "").strip()).lower() for l in x]
    return x

def main(N_TRIALS_PER_COMBO=1):
    results = [] 
    
    grid_search = {
        'n_examples': [4],
        'temperature': [0.5, 0.7, 0.9],
        'frequency_penalty': [0.5, 1, 1.5],
        'presence_penalty': [0.5, 1, 1.5]
    }
    
    logging.info(f"n_trials_combo: {N_TRIALS_PER_COMBO}")

    all_combinations = list(itertools.product(
        grid_search['n_examples'],
        grid_search['temperature'],
        grid_search['frequency_penalty'],
        grid_search['presence_penalty'],
    ))
    logging.info(f"grid_search parameters: {grid_search}, len {len(all_combinations)}")
    logging.info(f"prompts: {PROMPTS}")
    logging.info(f"AUT ITEMS: {AUT_ITEMS}")
    print(len(PROMPTS))
    print(len(AUT_ITEMS))
    print(len(all_combinations))
    print(N_TRIALS_PER_COMBO)
    total_requests = len(PROMPTS) * len(AUT_ITEMS) * len(all_combinations) * N_TRIALS_PER_COMBO
    logging.info(f"TOTAL REQUESTS: {total_requests}")


    counter = 0

    for aut_item in AUT_ITEMS:
        for prompt_name, prompt_base in PROMPTS.items():
            for combination in all_combinations:
                for trial in range(N_TRIALS_PER_COMBO):
                    n_examples, temperature, frequency_penalty, presence_penalty = combination
                    examples = get_examples(example_df, aut_item, n_examples, seed=counter)
                    generated_response = handle_prompt(prompt_base,
                                                       object_name="a " + aut_item,
                                                       examples=examples,
                                                       n_examples=n_examples,
                                                       temperature=temperature,
                                                       frequency_penalty=frequency_penalty,
                                                       presence_penalty=presence_penalty)

                    row = {
                        'prompt_condition': prompt_name,
                        'trial_no':trial,
                        'examples': examples,
                        'output_responses': generated_response,
                        'n_examples': n_examples,
                        'temperature': temperature,
                        'frequency_penalty': frequency_penalty,
                        'presence_penalty': presence_penalty
                    }
                    results.append(row)
                    if counter % 100 == 0:
                        logging.info(f"{counter} of {total_requests}")
                    counter+=1
    df = pd.DataFrame(results)
    df.to_csv("results.csv")
main()



3
1
27
1
1. Build a decorative planter box - Place several bricks in a square or rectangular shape and fill with soil and plants to create an attractive garden feature.

2. Make a paperweight - Use a brick to keep important documents from blowing away or getting lost. 

3. Create a doorstop - Place a brick at the bottom of the door to keep it from swinging open unexpectedly. 

4. Create a stepping stone - Paint the brick with unique designs and place it in your garden as an eye-catching stepping stone path.


In [107]:
results_df = pd.read_csv(csv_file)


In [108]:
results_df

,prompt_condition,examples,output_responses,n_examples,temperature,frequency_penalty,presence_penalty
0,zero_shot,"['small chair', 'BREAK A NUT', 'BUILD', 'A way...",.\n\n1. Use a brick as an outdoor planter to h...,4,0.7,1.5,1.0
1,implicit,"['Sharpen tools', 'headrest', 'hold a car in p...",- Use as a doorstop \n- Weight for stretching ...,4,0.7,1.5,0.5
2,explicit,"['carpet', 'build homes in Africa', 'canvas to...",.\n- bookend \n- birdbath base \n- stepping st...,4,0.7,1.5,1.5
3,zero_shot,"['Hobo House', 'hold items', 'keep a homeless ...",1. Turn a box into a tiny library by putting b...,4,0.9,0.5,0.5
4,implicit,"['alternative to painters paper', 'draw eyes a...",1. Create a puppet theater \n2. Use as a canva...,4,0.5,1.5,1.5
5,explicit,"['get dressed', 'climbing', 'planter', 'paint ...",.\n\n- puppet theatre \n- storage for toys \n-...,4,0.5,0.5,1.0
6,zero_shot,"['command someone by threat', 'cleaning out th...",1. Create a unique piece of art by carving sha...,4,0.5,1.5,0.5
7,implicit,"['art by hanging it on wall', 'wall decoration...",1. Carving wood sculptures \n2. Crafting jewel...,4,0.5,0.5,0.5
8,explicit,"['razor to shave', 'stab in wall and hang pict...",.\n\n- Use the tip to open a bottle \n- Cut th...,4,0.5,1.5,1.5


In [104]:
import sys
!{sys.executable} -m pip install tenacity

  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)


In [105]:
import tenacity
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)

In [111]:
example_df['prompt'].value_counts()

brick         5162
box           2842
knife         2163
rope          2080
paperclip     1385
bottle         839
book           487
table          461
pants          443
tire           412
fork           407
ball           393
spoon          386
pencil         384
lightbulb      383
shoe           382
sock           380
hat            380
toothbrush     379
shovel         339
backpack        34
Name: prompt, dtype: int64

In [117]:
for x, y in PROMPTS.items():
    print(y)

What are creative uses for [OBJECT_NAME]? The goal is to come up with a creative idea, which is an idea that strikes people as clever, unusual, interesting, uncommon, humorous, innovative, or different. List [N] creative uses for [OBJECT_NAME]
What are creative uses for [OBJECT_NAME]? Here are example creative uses: [EXAMPLES] Based on the examples, list [N] creative uses for [OBJECT_NAME] that sounds like the examples.
What are creative uses for [OBJECT_NAME]? Here are example creative uses: [EXAMPLES] Carefully study the examples and their style, then list [N] creative uses for [OBJECT_NAME] that resemble the given examples. Match the style, length, and complexity of the creative ideas in the examples


In [118]:
   all_combinations = list(itertools.product(
        grid_search['n_examples'],
        grid_search['temperature'],
        grid_search['frequency_penalty'],
        grid_search['presence_penalty'],
    ))

In [120]:
len(all_combinations)

27